In [1]:
import pandas as pd
import yfinance as yf
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import logging
import itertools
from config import STOCKS_TO_TEST, TEST_YEARS, PERIOD

# Suppress verbose logging from Prophet
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

c:\Users\hemva\Documents\Programs\FinSight\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Define a grid of parameters to test ---
param_grid = {  
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [1.0, 10.0, 20.0],
}
# Create all possible combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

results = []

print("--- Starting Tuned Prophet Backtesting on Portfolio ---")

for ticker in STOCKS_TO_TEST:
    try:
        print(f"\nProcessing and tuning for {ticker}...")
        # --- Fetch Data ---
        data = yf.Ticker(ticker).history(period=PERIOD)
        data.reset_index(inplace=True)
        df_prophet = data[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
        df_prophet['ds'] = df_prophet['ds'].dt.tz_localize(None)

        # --- Train-Test Split ---
        split_date = df_prophet['ds'].max() - pd.DateOffset(years=TEST_YEARS)
        train_df = df_prophet[df_prophet['ds'] <= split_date]
        test_df = df_prophet[df_prophet['ds'] > split_date]

        best_params = {}
        best_rmse = float('inf')

        # --- Grid Search Loop ---
        for params in all_params:
            # Train model with the current set of parameters
            model = Prophet(**params, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
            model.fit(train_df)
            
            # Predict and evaluate
            future = model.make_future_dataframe(periods=len(test_df))
            predictions = model.predict(future)
            predictions = predictions[predictions['ds'] > split_date]
            
            y_true = test_df['y'].values
            y_pred = predictions['yhat'].values
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))

            # If this set of parameters is better, save them
            if rmse < best_rmse:
                best_rmse = rmse
                best_params = params

        # --- Retrain with best params to get final MAE ---
        print(f"Best params for {ticker}: {best_params}")
        final_model = Prophet(**best_params, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
        final_model.fit(train_df)
        future = final_model.make_future_dataframe(periods=len(test_df))
        predictions = final_model.predict(future)
        predictions = predictions[predictions['ds'] > split_date]
        y_true = test_df['y'].values
        y_pred = predictions['yhat'].values
        
        final_mae = mean_absolute_error(y_true, y_pred)
        
        results.append({"Ticker": ticker, "MAE": final_mae, "RMSE": best_rmse})
        print(f"Processed {ticker} - Best RMSE: ${best_rmse:.2f}, Final MAE: ${final_mae:.2f}")

    except Exception as e:
        print(f"Could not process {ticker}. Error: {e}")

print("\n--- Tuned Prophet Backtesting Complete ---")

--- Starting Tuned Prophet Backtesting on Portfolio ---

Processing and tuning for AAPL...


17:25:29 - cmdstanpy - INFO - Chain [1] start processing
17:25:29 - cmdstanpy - INFO - Chain [1] done processing
17:25:29 - cmdstanpy - INFO - Chain [1] start processing
17:25:29 - cmdstanpy - INFO - Chain [1] done processing
17:25:29 - cmdstanpy - INFO - Chain [1] start processing
17:25:30 - cmdstanpy - INFO - Chain [1] done processing
17:25:30 - cmdstanpy - INFO - Chain [1] start processing
17:25:30 - cmdstanpy - INFO - Chain [1] done processing
17:25:31 - cmdstanpy - INFO - Chain [1] start processing
17:25:31 - cmdstanpy - INFO - Chain [1] done processing
17:25:32 - cmdstanpy - INFO - Chain [1] start processing
17:25:32 - cmdstanpy - INFO - Chain [1] done processing
17:25:33 - cmdstanpy - INFO - Chain [1] start processing
17:25:33 - cmdstanpy - INFO - Chain [1] done processing
17:25:34 - cmdstanpy - INFO - Chain [1] start processing
17:25:34 - cmdstanpy - INFO - Chain [1] done processing
17:25:35 - cmdstanpy - INFO - Chain [1] start processing
17:25:36 - cmdstanpy - INFO - Chain [1]

Best params for AAPL: {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 1.0}


17:25:36 - cmdstanpy - INFO - Chain [1] done processing


Processed AAPL - Best RMSE: $15.53, Final MAE: $12.91

Processing and tuning for MSFT...


...

Processed F - Best RMSE: $0.81, Final MAE: $0.66

Processing and tuning for NVDA...


17:26:10 - cmdstanpy - INFO - Chain [1] start processing
17:26:10 - cmdstanpy - INFO - Chain [1] done processing
17:26:11 - cmdstanpy - INFO - Chain [1] start processing
17:26:11 - cmdstanpy - INFO - Chain [1] done processing
17:26:11 - cmdstanpy - INFO - Chain [1] start processing
17:26:11 - cmdstanpy - INFO - Chain [1] done processing
17:26:12 - cmdstanpy - INFO - Chain [1] start processing
17:26:12 - cmdstanpy - INFO - Chain [1] done processing
17:26:13 - cmdstanpy - INFO - Chain [1] start processing
17:26:13 - cmdstanpy - INFO - Chain [1] done processing
17:26:13 - cmdstanpy - INFO - Chain [1] start processing
17:26:14 - cmdstanpy - INFO - Chain [1] done processing
17:26:14 - cmdstanpy - INFO - Chain [1] start processing
17:26:15 - cmdstanpy - INFO - Chain [1] done processing
17:26:15 - cmdstanpy - INFO - Chain [1] start processing
17:26:16 - cmdstanpy - INFO - Chain [1] done processing
17:26:16 - cmdstanpy - INFO - Chain [1] start processing
17:26:17 - cmdstanpy - INFO - Chain [1]

Best params for NVDA: {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 1.0}


17:26:18 - cmdstanpy - INFO - Chain [1] done processing


Processed NVDA - Best RMSE: $19.98, Final MAE: $17.57

--- Tuned Prophet Backtesting Complete ---


In [4]:
# --- Display Final Results ---
results_df = pd.DataFrame(results)
print("\n--- Prophet Model Performance Summary ---")
print(results_df.to_string())

# Print the average performance
print("\n--- Average Performance ---")
print(results_df.mean(numeric_only=True))


--- Prophet Model Performance Summary ---
  Ticker        MAE       RMSE
0   AAPL  12.914921  15.531923
1   MSFT  30.924863  37.471464
2    JPM  13.157970  16.031771
3    JNJ   7.339766   9.548953
4      F   0.664972   0.806029
5   NVDA  17.573645  19.983243

--- Average Performance ---
MAE     13.762689
RMSE    16.562230
dtype: float64
